In [102]:
import pandas as pd
import re
import time
from datetime import datetime
import os

In [148]:
##參數說明:
'''前兩個是起迄日期，填到[DAY]，格式為XXXX(year)-XX(month)-XX(day)
   trade_type為交易類別，please填入list
   condition為你要看大於等於or小於，可選擇的有
      1.大於等於
      2.大於
      3.等於
      4.小於
      5.小於等於
   threshold為要看的門檻(填台幣)
   Agent 同樣填入list
   output_filename是期望的檔名，預設為最近一次查詢
   是否為台股基金預設是None，有需要就Y 不需要(想看非台股)則N'''

current_path = os.getcwd()
def query_proccess(start_date, end_date, trade_type=['ARSP', 'CRSP', 'LRSP', 'ROBO RSP', 'ROBO(單筆)', 'RSP', 'RSP30',
      '單筆申購', '單筆申購(EC)'], condition=None, threshold=None, Agent=None, 是否為台股基金 = None, output_filename = '最近一次查詢',
      基金簡稱_list = None):
   ###先找到橫跨月份
   all_month_need   = []
   start_date_month = re.search(r'(\d+-)(\d+)(-\d+)',start_date).group(1) + re.search(r'(\d+-)(\d+)(-\d+)',start_date).group(2)
   經過的月份        = [x.strftime("%Y-%m") for x in pd.date_range(start_date, end_date, freq='MS')]
   #end_date_month   = re.search(r'(\d+-)(\d+)(-\d+)',end_date).group(1) + re.search(r'(\d+-)(\d+)(-\d+)',end_date).group(2)
   all_month_need.append(start_date_month)
   all_month_need.extend(經過的月份)
   #all_month_need.append(end_date_month)
   ###將月份變檔案
   all_file_need    = [mon+'.csv' for mon in all_month_need]

   print(all_file_need)
   
   #用df來裝這些東西，迭代要看的檔案，如果是最前面最後面的檔案，那要記得界線
   dataframe        = pd.DataFrame({})
   for index, file in enumerate(all_file_need):
      current_file  = current_path+'\\'+file

      if index == 0:
         #print(current_file)
         temp_df = pd.read_csv(current_file)
         temp_df['日期'] = pd.to_datetime(temp_df['日期'])
         new_temp_df = temp_df[temp_df['日期']>= datetime.strptime(start_date, "%Y-%m-%d")]


      elif index == len(all_file_need) - 1:
         #print(current_file)
         temp_df = pd.read_csv(current_file)
         temp_df['日期'] = pd.to_datetime(temp_df['日期'])
         new_temp_df = temp_df[temp_df['日期']<= datetime.strptime(end_date, "%Y-%m-%d")]

      else:
         #print(current_file)
         new_temp_df = pd.read_csv(current_file)
         new_temp_df['日期'] = pd.to_datetime(new_temp_df['日期'])
      
      dataframe = dataframe.append(new_temp_df)

   #定義查詢條件式
   if condition != None and threshold != None:
      if condition == '大於':
         dataframe = dataframe[(dataframe['交易類別'] in trade_type) & (dataframe['金額(台幣)'] >  threshold)]
      elif condition == '小於':
         dataframe = dataframe[(dataframe['交易類別'] in trade_type) & (dataframe['金額(台幣)'] <  threshold)]
      elif condition == '等於':
         dataframe = dataframe[(dataframe['交易類別'] in trade_type) & (dataframe['金額(台幣)'] == threshold)]
      elif condition == '大於等於':
         dataframe = dataframe[(dataframe['交易類別'] in trade_type) & (dataframe['金額(台幣)'] >= threshold)]
      elif condition == '小於等於':
         dataframe = dataframe[(dataframe['交易類別'] in trade_type) & (dataframe['金額(台幣)'] <= threshold)]
   else:
      pass
   


   if Agent != None:
      dataframe = dataframe[dataframe['Agent'] in Agent]
   else:
      pass

   if 是否為台股基金 == 'Y':
      dataframe = dataframe[dataframe['是否為台股基金']=='Y']
   elif 是否為台股基金 == 'N':
      dataframe = dataframe[dataframe['是否為台股基金']=='N']
   else:
      pass

   if 基金簡稱_list == None:
      pass
   else:
      dataframe = dataframe[dataframe['基金簡稱'].isin(基金簡稱_list)]


   

   output_file  = current_path+'\\'+output_filename+'.csv'
   dataframe.to_csv(output_file,encoding='utf-8-sig',index=False,header=True)
   return 'Task Done!'

In [150]:
query_proccess('2021-12-24','2022-02-06')

['2021-12.csv', '2022-01.csv', '2022-02.csv']


'Task Done!'